<a href="https://colab.research.google.com/github/feliciapedwards/TensorFlow-with-Laurence-Moroney/blob/master/Course1Wk3_Convolutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Improving our model using Convolutions.
For review, lets take a look at our model from last week.

In [2]:
import tensorflow as tf

mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images / 255.0
test_images=test_images / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=5)

test_loss = model.evaluate(test_images, test_labels)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 4s 67us/sample - loss: 0.5024 - acc: 0.8231
Epoch 2/5
60000/60000 [==============================] - 4s 67us/sample - loss: 0.3777 - acc: 0.8644
Epoch 3/5
60000/60000 [==============================] - 4s 67us/sample - loss: 0.3398 - acc: 0.8750
Epoch 4/5
60000/60000 [==============================] - 4s 67us/sample - loss: 0.3117 - acc: 0.8865
Epoch 5/5
10000/10000 [==============================] - 0s 36us/sample - loss: 0.3639 - acc: 0.8685


Our training data has 89% accuracy while our test data has 87% accuracy. How do we improve this? One way is through convolutions.

Convolutions try to pinpoint specific details of the image that help distinguish them. We add some layers to create convolution before the dense layers. This refines the information that is passed down to the dense layers, hopefully making it more concentrated and focused.

In [3]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

model.fit(training_images, training_labels, epochs=5)

test_loss = model.evaluate(test_images, test_labels)

1.15.0
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               204928    
_________________________________________________________________
dense_5 (Dense)              (None, 10)        

Here we had to shape both our training and test data to (60000, 28, 28, 1) and (10000, 28, 28, 1). This is critical, otherwise we'll get an error since the Convolutions wont be able to recognize the shape. 

The first layer "tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1))" in our model instructs keras to generate 64 filters. These filters are 3 by 3, their activation is relu, which means we only keep our 0 or larger values, and  the input shape 28 by 28 is maintained. Since our images are gray scale, we use "1" for a single byte for color depth.

The second layer "tf.keras.layers.MaxPooling2D(2, 2)," creates a pooling layer. It's called a max-pooling because we're taking the maximum value. We're specifying a two-by-two pool, so for every four pixels, the largest value will be the values that is passed on. This reduces the image size by a quarter.

The second and third layer are created so that the network can learn an additional set of convolutions, and we pool again to reduce the size again. So when all is said and done, the images have been quartered twice.

The train data accuracy is 93% and the test data accuracy is 91%. 

Lets change the number of convolutons from 64 to 32 and see what happens.

In [4]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

model.fit(training_images, training_labels, epochs=5)

test_loss = model.evaluate(test_images, test_labels)

1.15.0
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               102528    
_________________________________________________________________
dense_7 (Dense)              (None, 10)        